# Combining the Financial Statements Data and the Combined Public Ministries Data

### Table of Contents
1. [Data Sources](#Data-Sources)   
2. [Purpose](#purpose)
3. [Tests](#tests)
4. [Combining Revenue and Spend](#creating-the-final-excel-with-sources-of-data)



This notebook assumes you’ve already completed Step 1 (Combining the Public Ministries Raw Data), which produces a single cleaned Excel file of ministry-level spending. 

## Data Sources

Here we pull together three inputs:

1. Combined public ministries data (combined_tbs_statements.xlsx) – the Step 1 output  containing both fiscal years with consistent column names.

2. Financial statement extract (financial_statement_data.xlsx)– figures manually transcribed from the PDF public accounts into financial_statement_data.xlsx, split into revenue and spending sheets.

3. Helper mapping table (helper_tables.xlsx) -  which bridges ministry names in the TBS dataset to the expense categories used in the financial statements 
(e.g., mapping several TBS ministries into the FS “Justice” bucket).

All data sources used here is stored in the "processed data" folder for ease of reference and ease of use.

## Purpose

The purpose of this notebook is to combine the financial statement extracted data with the public ministries data.

A few notes:

- Different Naming Conventions: As alluded in the above, the naming conventions for the ministries do not match the naming convention in the Financial Statement extract. We need to ensure that we are able to match them up exactly - which is the purpose of the helper mapping table.

- Different totals between the data sources: When we aggregate the public ministries data, the totals do not fully line up with the totals reported in the financial statements. To reconcile them exactly, we create an "unreported" category that captures the residual difference for each financial statement category so that TBS + unreported = financial statement totals.

In [1]:
import pandas as pd
import numpy as np

# --- 1. File paths ---
combined_tbs_path = "./processed_data/combined_tbs_statements.xlsx"
financial_statement_path = "./processed_data/financial_statement_data.xlsx"
helper_tables_path = "./processed_data/helper_tables.xlsx"

In [2]:
# --- 2. Load the datasets ---
df_tbs = pd.read_excel(combined_tbs_path)
df_fs_spending = pd.read_excel(financial_statement_path, sheet_name="spending")
df_helpers = pd.read_excel(helper_tables_path)

In [3]:
df_helpers.head()

,tbs_ministry_mapping,fs_expenses_mapping
0,"Agriculture, Food and Rural Affairs",Other Programs
1,Attorney General,Justice
2,Cabinet Office,Other Programs
3,"Children, Community and Social Services",Children's and Social Services
4,Citizenship and Multiculturalism,Other Programs


In [4]:
df_tbs = df_tbs.merge(
    df_helpers[["tbs_ministry_mapping", "fs_expenses_mapping"]],
    left_on="Ministry Name",
    right_on="tbs_ministry_mapping",
    how="left"
)

In [5]:
df_tbs["Amount_2023_24"] = df_tbs["Amount_2023_24"]/1000000
df_tbs["Amount_2024_25"] = df_tbs["Amount_2024_25"]/1000000

In [6]:
agg_tbs = df_tbs.groupby("fs_expenses_mapping")[["Amount_2023_24", "Amount_2024_25"]].sum().reset_index()

In [7]:
agg_fs = df_fs_spending[["sankey_2", "2023_24", "2024_25"]]

In [8]:
agg_fs_renamed = agg_fs.rename(columns={
    "sankey_2": "category",
    "2023_24": "FS_2023_24",
    "2024_25": "FS_2024_25"
})

agg_tbs_renamed = agg_tbs.rename(columns={
    "fs_expenses_mapping": "category",
    "Amount_2023_24": "TBS_2023_24",
    "Amount_2024_25": "TBS_2024_25"
})

In [9]:
combined = agg_fs_renamed.merge(
    agg_tbs_renamed,
    on="category",
    how="outer"
)

combined.fillna(0, inplace=True)

In [10]:
combined["Diff_2023_24"] = combined["FS_2023_24"] - combined["TBS_2023_24"]
combined["Diff_2024_25"] = combined["FS_2024_25"] - combined["TBS_2024_25"]

In [11]:
df_tbs["sankey_4"] = np.where(
    df_tbs["Account Details (Expense/Asset Details)"] != "No Value",
    df_tbs["Account Details (Expense/Asset Details)"],
    df_tbs["Standard Account (Expense/Asset Name)"]
)

In [12]:
final_expenses = df_tbs[["fs_expenses_mapping", "tbs_ministry_mapping", "Activity / Item", "sankey_4", "Amount_2023_24", "Amount_2024_25"]]

In [13]:
final_expenses.rename(columns={
    "tbs_ministry_mapping": "sankey_2",
    "Activity / Item": "sankey_3",
}, inplace=True)

C:\Users\Boris\AppData\Local\Temp\ipykernel_21368\2015400909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expenses.rename(columns={


In [14]:
# Create unreported rows from the difference data
unreported = combined[["category", "Diff_2023_24", "Diff_2024_25"]].copy()

unreported = unreported.rename(columns={
    "category": "fs_expenses_mapping",
    "Diff_2023_24": "Amount_2023_24",
    "Diff_2024_25": "Amount_2024_25"
})

# Set sankey_2 to the category, sankey_3 to "unreported"
unreported["sankey_2"] = unreported["fs_expenses_mapping"]
unreported["sankey_3"] = "unreported"
unreported["sankey_4"] = "unreported"  # or leave as empty string "" if you prefer

# Reorder columns to match final_expenses
unreported = unreported[["fs_expenses_mapping", "sankey_2", "sankey_3", "sankey_4", "Amount_2023_24", "Amount_2024_25"]]

# Append to final_expenses
final_expenses = pd.concat([final_expenses, unreported], ignore_index=True)

In [15]:
# Sum final_expenses by fs_expenses_mapping
validation = final_expenses.groupby("fs_expenses_mapping")[["Amount_2023_24", "Amount_2024_25"]].sum().reset_index()

# Compare against financial statement values
comparison = validation.merge(
    agg_fs_renamed,
    left_on="fs_expenses_mapping",
    right_on="category",
    how="outer"
)

# Check the differences (should all be 0 or very close to 0)
comparison["Check_2023_24"] = comparison["Amount_2023_24"] - comparison["FS_2023_24"]
comparison["Check_2024_25"] = comparison["Amount_2024_25"] - comparison["FS_2024_25"]

comparison[["fs_expenses_mapping", "Amount_2023_24", "FS_2023_24", "Check_2023_24", 
            "Amount_2024_25", "FS_2024_25", "Check_2024_25"]]

,fs_expenses_mapping,Amount_2023_24,FS_2023_24,Check_2023_24,Amount_2024_25,FS_2024_25,Check_2024_25
0,Children's and Social Services,19412.0,19412,0.0,20736.0,20736,0.0
1,Education,38810.0,38810,0.0,40059.0,40059,0.0
2,Health,85458.0,85458,0.0,91631.0,91631,0.0
3,Interest and Other Debt Servicing Charges,14461.0,14461,0.0,15122.0,15122,0.0
4,Justice,6037.0,6037,0.0,7224.0,7224,0.0
5,Other Programs,32255.0,32255,0.0,38333.0,38333,0.0
6,Postsecondary Education,13235.0,13235,0.0,14146.0,14146,0.0


## Tests

This code is a sanity check to ensure that we have done our all aggregations correctly by checking that the total final expenses equates back to the financial statement raw data.


In [16]:
# Quick totals check

diff_2023_24 = final_expenses["Amount_2023_24"].sum() - agg_fs_renamed["FS_2023_24"].sum()
diff_2024_25 = final_expenses["Amount_2024_25"].sum() - agg_fs_renamed["FS_2024_25"].sum()
print("Final Expenses Total 2023-24:", final_expenses["Amount_2023_24"].sum(), "vs Financial Statement Total 2023-24:", agg_fs_renamed["FS_2023_24"].sum())
print("Total Difference:", diff_2023_24)

print("Final Expenses Total 2024-25:", final_expenses["Amount_2024_25"].sum(), "vs Financial Statement Total 2024-25:", agg_fs_renamed["FS_2024_25"].sum())
print("Total Difference:", diff_2024_25)

#if the difference for 2023_24 and 2024_25 is 0, then we pass the tests
if diff_2023_24 == 0 and diff_2024_25 == 0:
    print("✅ Passed all the tests ✅")
else:
    # throw an error if the tests fail
    print("❌ Failed some of the tests ❌")
    raise ValueError("Failed some of the tests")



Final Expenses Total 2023-24: 209668.0 vs Financial Statement Total 2023-24: 209668
Total Difference: 0.0
Final Expenses Total 2024-25: 227251.0 vs Financial Statement Total 2024-25: 227251
Total Difference: 0.0
✅ Passed all the tests ✅


## Creating the final excel with sources of data

Now that we have combined the expenses in the financial statement with the public ministries data, we need to combine this with the revenue data to create our final excel data.

In [17]:
final_expenses.drop(columns=["fs_expenses_mapping"], inplace=True)
final_expenses["sankey_1"] = "spending"
final_expenses.rename(columns={
    "Amount_2023_24": "2023_24",
    "Amount_2024_25": "2024_25"
}, inplace=True)
final_expenses["Source"] = "TBS Public Accounts Ministry Statement CSV's"
final_expenses.to_excel("final_expenses.xlsx", index=False)


In [18]:
df_fs_revenue = pd.read_excel(financial_statement_path, sheet_name="revenue")
df_fs_revenue.rename(columns={
    "sankey_2": "sankey_2",
    "sankey_3": "sankey_3",
    "sankey_4": "sankey_4",
    2023: "2023_24",
    2024: "2024_25"
}, inplace=True)

In [19]:
# append the revenue data with the expense data - matching the columns
df_final = pd.concat([df_fs_revenue, final_expenses], ignore_index=True)

# if sankey_4 is unreported and sankey_3 is unreported, say sankey_4 is a NaN value
df_final["sankey_4"] = np.where(
    (df_final["sankey_4"] == "unreported") & (df_final["sankey_3"] == "unreported"),
    np.nan,
    df_final["sankey_4"]
)

# normalizing the data to the billions:
df_final["2023_24"] = df_final["2023_24"]/1000
df_final["2024_25"] = df_final["2024_25"]/1000


df_final.to_excel("./processed_data/final_data.xlsx", index=False)



